**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 3 - Cross link User and Project accounts

1 - Load required modules

In [1]:
from new_helper import *
from super_helper import *
from loguru import logger
import pickle

2 - Set variables and values

In [2]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 0

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

#for the index system
index_path = "indexdir"
index = open_dir(index_path)


4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [3]:
hash = create_detail_contract()
integration_helpers.get_engine_receipts_result(hash)

2025-01-19 23:54:24.190 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-19 23:54:24.196 | INFO     | iroha_helper:create_detail_contract:55 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-19 23:54:24.197 | INFO     | iroha_helper:create_detail_contract:55 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-19 23:54:25.851 | INFO     | iroha_helper:create_detail_contract:55 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-19 23:54:25.879 | INFO     | iroha_helper:create_detail_contract:55 - ('COMMITTED', 5, 0)
2025-01-19 23:54:25.881 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-19 23:54:25.884 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-19 23:54:25.903 | INFO     | integration_helpers:get_engine_receipts_result:92 - 
2025-01-19 23:54:25.904 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


5 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` (see item 2) with a entry number related to the position of an existing object in `datasets/accounts.json` and `datasets/projects.json`. Example: `json_ld_index = 2`, selects the second object in each json file.

6 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [4]:
address = integration_helpers.get_engine_receipts_address(hash)

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-01-19 23:54:25.919 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-19 23:54:25.939 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-19 23:54:25.940 | INFO     | iroha_helper:tracer:265 - 	Entering "set_account_detail"
2025-01-19 23:54:25.962 | INFO     | iroha_helper:set_account_detail:317 - None
2025-01-19 23:54:25.965 | INFO     | iroha_helper:set_account_detail:319 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-19 23:54:28.911 | INFO     | iroha_helper:set_account_detail:319 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-19 23:54:28.940 | INFO     | iroha_helper:set_account_detail:319 - ('COMMITTED', 5, 0)
2025-01-19 23:54:28.942 | INFO     | iroha_helper:tracer:267 - 	Leaving "set_account_detail"
2025-01-19 23:54:28.944 | INFO     | iroha_helper:tracer:265 - 	Entering "set_account_detail"
2025-01-19 23:54:28.962 | INFO     | iroha_helper:set_account_detail:317 - None
2025-01-19 23:54:28.9

7 - Queries Iroha 1 for the User account and exposes the variable `project_id` from user details.

In [5]:
#Query - GetAccountDetail

from iroha_helper import get_account_detail

user_details = get_account_detail(f"{user_account['account_id']}")

logger.info(f'User Account id = {user_account}, {user_details}')

# # Process the account details response in JSON format
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object

# # Exposes the project_id 
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(f"Project Id = {project_id}")

2025-01-19 23:54:32.033 | INFO     | __main__:<module>:7 - User Account id = {'account_id': 'dreamy_johnson@test'}, { "admin@test" : { "linked_project" : "56557@test", "user_json_ld_cid" : "QmcVgzyDbnHEPFv6dbRGjgmcip4ZCbd7hfryXv6zYsif5m" } }
2025-01-19 23:54:32.035 | INFO     | __main__:<module>:14 - Project Id = 56557@test


8 - Saves `project_id` in the local file sytem for use in Notebook 4

In [6]:
# Save project_id to a file
try:
    # project_id = "example_project_id"  # Ensure project_id is defined
    file_path = "temp/project_id.pkl"
    
    with open(file_path, "wb") as f:
        pickle.dump(project_id, f)
    
    logger.success(f"Successfully saved project_id to {file_path}.")
except Exception as e:
    logger.error(f"Failed to save project_id to file. Error: {e}")



2025-01-19 23:54:32.048 | SUCCESS  | __main__:<module>:9 - Successfully saved project_id to temp/project_id.pkl.
